### Get cpu information on linux

Commands  
`lscpu`  
`cat /proc/cpuinfo`  

```
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               94
Model name:          Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz
Stepping:            3
CPU MHz:             800.055
CPU max MHz:         3500,0000
CPU min MHz:         800,0000
BogoMIPS:            5184.00
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            6144K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq dtes64 monitor ds_cpl vmx est tm2 ssse3 sdbg fma cx16 xtpr pdcm pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb invpcid_single pti ssbd ibrs ibpb stibp tpr_shadow vnmi flexpriority ept vpid fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx rdseed adx smap clflushopt intel_pt xsaveopt xsavec xgetbv1 xsaves dtherm ida arat pln pts hwp hwp_notify hwp_act_window hwp_epp flush_l1d
```

Flags: indicates capabilities of the processor
- mmx
- lm (long mode) (probably indicates x64 support)

### Registers

Besides the x86 registers extended (EAX => RAX)  
There are 7 brand new register on x64 archs: R8 to R15  
They are divided like:  
r8 (64) => r8d (lower 32) => r8w (lower 16) => r8b (lower 8)    
r8  = full qword reg  
r8d = lower dword  
r8w = lower word    
r8b = lower byte     
Upper 8 is not accesible for these registers  

### Syscall convention

RAX = Syscall number  
RDI = 1st arg  
RSI = 2nd arg  
RDX = 3rd arg  
R10 = 4th arg  
R8  = 5th arg  
R9  = 6th arg

RAX = Return value will be placed here

Example for write syscall:  
> ssize_t write(int fd, const void *buf, size_t count)   

Arguments:
- syscall number (1) => RAX
- file descriptor (1 == stdout) => RDI
- string pointer => RSI
- length of string => RDX

### Reduce instruction length

The smaller the register, the shorter the instruction  
Because the entire value must be hardcoded into the instruction for bigger registers  

Examples 64bit vs 8bit register results in 9bytes vs 2bytes long instructions:  
> rasm2 "mov rax, 1"  
> 48c7c001000000

> rasm2 "mov al, 1"  
> b001


### Shellcode should be env independant
We need to cleanup every register to be used because we can't ensure it won't be filled with garbage  


### NASM details  

Defining data (defines a label which represents the address at which the data is):
db = define byte  
dw = define word  
dd = define double  
dq = define quad  
`dataLabel db 0xAA, 0xBB, 0xCC`  

Loading data address into register:   
`mov rax, dataLabel`  

Loading data value into register:  
`mov rax, [dataLabel]`  

Define uninitialized data:  
This needs to be defined in the .bss section  
`buffer: resb 64 ; reserve 64 bytes`  
`wordvar: resw 1 ; reserve a word`  

`$ = current line`  
`$$ = beginning of the current section`  

equ = equals  


### GDB  

https://beej.us/guide/bggdb/  

enable TUI mode with option `-tui`  
use gdb-dashboard? peda?  

#### commands
```
disassembly-flavor intel   
layout asm  
layout regs  

run  
break <main/_start/etc>
info variables  
ni  
x/1xb &variableName
x/1xb 0x400174

x/8xb  ; show as bytes
x/16xb ; show as words
x/32xb ; show as dwords
x/64xb ; show as qwords

x/s  ; show as a string
```

### String operations  

##### Compare   
###### - memory vs register  
scas(b/w/d/q) : Compares string referenced by RDI agasint al/ax/eax/rax
(scasb for byte, scasw for word & so on...)  
example: 
```
mov rax, [var1]
lea rdi, [var2]
scasq
; check ZF here, if ZF == 1 then they are equal
```
###### - memory vs memory
cmps(b/w/d/q) : Comprares string referenced by RSI vs referecenced by RDI  
example:  
```
lea rdi, [var1]
lea rsi, [var2]
cmpsq
; check ZF here, if ZF == 1 then they are equal
```

##### Load (Copy from memory addres loaded into RDI to RAX)
lods(b/w/d/q)  

##### Store (Copy from RAX register to memory location loaded on RDI)
stos(b/w/d/q)  

##### Move  (Copy from memory to memory)
movs(b/w/d/q)  
Takes the value in the __DIRECTION FLAG__:
- DF == 0 => the addreses are incremented each round
- DF == 1 => the addreses are decremented each round
- Remember to clear the direction flag (**cld**) or set it if needed  
Example:   
```
cld
lea rsi, [var1]
lea rdi, [var2]
movsq ; copy from @var1 (direction incremental [DF == 0]) to @var2
``` 
